In [1]:
import sys, os

data_lib = os.path.abspath('data')
if data_lib not in sys.path:
    sys.path.append(data_lib)
import os
import os.path as osp
import numpy as np
import configparser
from models.classifiers import BinaryClassifier
from models.deep_neural_network import *
import pandas as pd
from sklearn.model_selection import LeaveOneGroupOut
from tqdm import tqdm
from data.dataset_loader import *
from data.result_utils import *
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import torch
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.svm import SVC

In [2]:
# %%
# -- Uncomment the dataset that you wanna load -- #
# dataset_name = 'AffectiveROAD'
dataset_name = 'WESAD'
# device = 'right'
device = 'wrist'
# physiological_signal_type = 'BVP_EDA'
physiological_signal_type = 'BVP_EDA_TEMP'
# physiological_signal_type = 'BVP'
# NORMALIZER = '_nonorm'
NORMALIZER = ''
# physiological_signal_type = 'EDA'
# dataset_name = 'DCU_NVT_EXP1'

# %%
WINDOW_SHIFT = 0.25
WINDOW_SIZE = 60
dp_manager = get_data_path_manager()

In [3]:
# %%
ds_loader = DatasetLoader(dataset_name, device, physiological_signal_type, WINDOW_SHIFT = WINDOW_SHIFT, WINDOW_SIZE = WINDOW_SIZE)

# %% [markdown]
# # Define stress detection strategies

# %%
# -- Uncomment the detection strategy that you wanna use to detect -- #
strategies = ['svm'] #, 'mlp']
# detection_strategy = 'logistic_regression'
# detection_strategy = 'random_forest'
# detection_strategy = 'svm'
# detection_strategy = 'mlp'
# detection_strategy = 'knn'


# %%
# SCORING = 'accuracy'
SCORING = 'balanced_accuracy'

In [4]:
def get_classifier(method: str):
    clf = None
    if method == 'random_forest':
        clf = RandomForestClassifier(n_estimators = 250, random_state = 0, n_jobs = -1, max_depth = 8, min_samples_leaf = 4, min_samples_split = 2, oob_score = True, bootstrap = True, class_weight = 'balanced')
    elif method == 'knn':
        clf = KNeighborsClassifier(n_neighbors = 5, weights = 'distance', n_jobs=-1)
    elif method == 'svm':
        clf = SVC(C = 10, kernel = 'rbf', gamma = 'scale', class_weight = 'balanced', random_state = 0)
    return clf
        

In [5]:
for train_index, test_index in tqdm(LeaveOneGroupOut().split(ds_loader.dataset, ds_loader.ground_truth, ds_loader.groups)):
    X_train, X_test = ds_loader.dataset[train_index], ds_loader.dataset[test_index]
    y_train, y_test = ds_loader.ground_truth[train_index], ds_loader.ground_truth[test_index]
    groups_test = ds_loader.groups[test_index][0]
    for strategy in strategies:
        pipeline = []
        if strategy in ['knn', 'svm']:
            pipeline.append(('scaler', StandardScaler()))
        clf = get_classifier(strategy)
        pipeline.append(('classifier', clf))
        pipeline = Pipeline(pipeline)
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_true = y_test
        acc = accuracy_score(y_true, y_pred)
        balanced_acc = balanced_accuracy_score(y_true, y_pred)
        print(f'Test subject {groups_test} --- Accuracy: {acc} --- Balanced Accuracy: {balanced_acc}')

1it [01:32, 92.06s/it]

Test subject S10 --- Accuracy: 0.7865999046256557 --- Balanced Accuracy: 0.8403268996513631


2it [03:09, 93.71s/it]

Test subject S11 --- Accuracy: 0.9677734375 --- Balanced Accuracy: 0.9660533568266017


3it [04:49, 95.42s/it]

Test subject S13 --- Accuracy: 0.9937683284457478 --- Balanced Accuracy: 0.9906480490672447


In [ ]:
# for detection_strategy in strategies:
#     detector_type = 'General'
#     print(f'--- RUNNING {detector_type} {detection_strategy} ---')
#     clf = BinaryClassifier(ds_loader.dataset, ds_loader.ground_truth, detection_strategy, basic_logo_validation = True, groups = ds_loader.groups, scoring = SCORING)
#     results = clf.exec_classifier() # Build classifier and return prediction results
#     print('------------------------------------------------------')

    # # %%
    # # Save results
    # result_helper = ResultUtils(dataset_name, device, physiological_signal_type, NORMALIZER = NORMALIZER, WINDOW_SHIFT = WINDOW_SHIFT, WINDOW_SIZE = WINDOW_SIZE)
    # result_helper.dump_result_to_csv(results, detection_strategy, detector_type)